# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from pathlib import Path

from cleanup import UniqueIDer, gen_result_df

# Load

### Set Paths

In [ ]:
pickle_folder = Path(r'jsl\OneDrive-2020-01-30')
# yaml_path = Path(r'jsl\jsl.yaml')
yaml_path = Path(r'jsl\PhotoOrg-JDL.yaml')

### Load from folder

In [ ]:
print(f'Loading pickle files from:\n{str(pickle_folder.resolve())}')

def load_and_print(file):
    print(f'Loading {file.stat().st_size / (10**6):.2f} MB from {file.name}')
    return pd.read_pickle(file)

df = pd.concat([load_and_print(f) for f in pickle_folder.glob('*.pkl') if 'jdl' not in f.name], sort=False)
df = df.reset_index(drop=True)

print(f'Loaded {df.shape[0]} files, {df.memory_usage().sum() / (10**6):.2f} MB DataFrame')

# Find Unique Files

Create the Object that's going to process the files from your YAML file, which has all the settings for file inclusion/exclusion

In [ ]:
uid = UniqueIDer.from_yaml(yaml_path, df)
uid.df.to_pickle(pickle_folder / 'jdl_filtered.pkl')

Process the files in groups using unique combinations of the values in columns
- `st_size`
- `suffix`
- `shortname`

The `suffix` and `shortname` columns are created during processing. You can also customize the set of columns used for grouping like this:  
`uid.flat_process(keys=[col1, col2, ...])`

In [ ]:
uid.process(priority_keyword='Lightroom')
'Some files were marked both dup and unique' if (uid.mask_u == uid.mask_d).any() else 'Processing worked'

# Save Unique Files

It will default to saving in the same folder as your YAML file

In [ ]:
uid.unique.to_csv(pickle_folder / 'jdl_unique.csv')
uid.unique.to_pickle(pickle_folder / 'jdl_unique.pkl')